In [ ]:
! pip install tensorflow_recommenders

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfr
import pprint

In [43]:
ratings = tfds.load("movielens/100k-ratings",split="train")
movies = tfds.load("movielens/100k-movies",split="train")

In [44]:
for x in ratings.take(2).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'bucketized_user_age': 25.0,
 'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'raw_user_age': 32.0,
 'timestamp': 875654590,
 'user_gender': True,
 'user_id': b'92',
 'user_occupation_label': 5,
 'user_occupation_text': b'entertainment',
 'user_rating': 2.0,
 'user_zip_code': b'80525'}


In [45]:
for x in movies.take(3).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}
{'movie_genres': array([4, 7]),
 'movie_id': b'1457',
 'movie_title': b'Love Is All There Is (1996)'}
{'movie_genres': array([1, 3]),
 'movie_id': b'500',
 'movie_title': b'Fly Away Home (1996)'}


In [62]:
## taking 2 features id and movie title from ratings

ratings = ratings.map(lambda x:{
    "movie_title":x["movie_title"],
    "user_id":x["user_id"],
   
})
movies = movies.map(lambda x:x["movie_title"])
                              

In [63]:
for x in ratings.take(5).as_numpy_iterator():
  pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138'}
{'movie_title': b'Strictly Ballroom (1992)', 'user_id': b'92'}
{'movie_title': b'Very Brady Sequel, A (1996)', 'user_id': b'301'}
{'movie_title': b'Pulp Fiction (1994)', 'user_id': b'60'}
{'movie_title': b'Scream 2 (1997)', 'user_id': b'197'}


In [64]:
for x in movies.take(5).as_numpy_iterator():
  pprint.pprint(x)

b'You So Crazy (1994)'
b'Love Is All There Is (1996)'
b'Fly Away Home (1996)'
b'In the Line of Duty 2 (1987)'
b'Niagara, Niagara (1997)'


In [66]:
# building vocabularies to convert data into integer indicies for embedding

user_ids_vocabulary= tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
movie_titles_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary.adapt(movies)

In [67]:
class TwoTowerMovielensModel(tfr.Model):
  ### prediction model
  def __init__(self, user_model: tf.keras.Model,movie_model: tf.keras.Model,task:tfr.tasks.Retrieval):
    super().__init__()
  
   # setting up movie and user model
    self.user_model = user_model
    self.movie_model = movie_model
     
    ## determining which task the model is going to perform..retrieval,ranking or post ranking
    self.task = task

  def compute_loss(self,features,training = False) -> tf.Tensor:
    user_embeddings = self.user_model(features['user_id'])
    movies_embeddings = self.movie_model(features["movie_title"])
    return self.task(user_embeddings,movies_embeddings)


In [69]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(),64)
])
movies_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(),64)
])
task = tfr.tasks.Retrieval(metrics=tfr.metrics.FactorizedTopK(movies.batch(128).map(movies_model)))

In [75]:
model = TwoTowerMovielensModel(user_model,movies_model,task)
model.compile(optimizer = tf.keras.optimizers.Adagrad(0.5))

In [76]:
model.fit(ratings.batch(4096),epochs=10)

Epoch 1/10
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 1.9000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0076 - factorized_top_k/top_50_categorical_accuracy: 0.0571 - factorized_top_k/top_100_categorical_accuracy: 0.1267 - loss: 33162.6486 - regularization_loss: 0.0000e+00 - total_loss: 33162.6486
Epoch 2/10
25/25 [==============================] - 29s 1s/step - factorized_top_k/top_1_categorical_accuracy: 2.1000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0057 - factorized_top_k/top_10_categorical_accuracy: 0.0169 - factorized_top_k/top_50_categorical_accuracy: 0.1140 - factorized_top_k/top_100_categorical_accuracy: 0.2258 - loss: 30838.6073 - regularization_loss: 0.0000e+00 - total_loss: 30838.6073
Epoch 3/10
25/25 [==============================] - 30s 1s/step - factorized_top_k/top_1_categorical_accuracy: 6.8000e-04 - factorized_top_k/top_5_categorical_a

In [ ]:
### predictions using brute force to search to set up retrival using the trained representation

In [77]:
index =tfr.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(movies.batch(100).map(lambda title:(title, model.movie_model(title))))

In [83]:
import numpy as np

In [86]:
for i in range(0,50):
  _, titles = index(np.array([str(i)]))
  print(f'top 3 recommendations for user{str(i)}:{titles[0,:3]}')

top 3 recommendations for user0:[b'That Darn Cat! (1997)' b'That Darn Cat! (1997)' b'Heavyweights (1994)']
top 3 recommendations for user1:[b'Brother Minister: The Assassination of Malcolm X (1994)'
 b'Doom Generation, The (1995)' b'Turbo: A Power Rangers Movie (1997)']
top 3 recommendations for user2:[b'3 Ninjas: High Noon At Mega Mountain (1998)' b'Promesse, La (1996)'
 b'Once Upon a Time... When We Were Colored (1995)']
top 3 recommendations for user3:[b'Critical Care (1997)' b'Spice World (1997)' b'Deep Rising (1998)']
top 3 recommendations for user4:[b'Assignment, The (1997)' b'Wonderland (1997)'
 b'Blues Brothers 2000 (1998)']
top 3 recommendations for user5:[b'Amityville Curse, The (1990)' b'Amityville II: The Possession (1982)'
 b"Amityville 1992: It's About Time (1992)"]
top 3 recommendations for user6:[b'Ponette (1996)' b'His Girl Friday (1940)' b'Daytrippers, The (1996)']
top 3 recommendations for user7:[b'Police Story 4: Project S (Chao ji ji hua) (1993)'
 b"Daniel Defoe's 